In [3]:
# install spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] ="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] ="/content/spark-3.2.4-bin-hadoop3.2"
import findspark
findspark.init()

spark-3.2.4-bin-hadoop3.2/
spark-3.2.4-bin-hadoop3.2/R/
spark-3.2.4-bin-hadoop3.2/R/lib/
spark-3.2.4-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/html/
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/html/R.css
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/html/00Index.html
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/help/
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/help/aliases.rds
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/help/AnIndex
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/help/SparkR.rdx
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/help/SparkR.rdb
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/help/paths.rds
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/INDEX
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.2.4-bin-hadoop3.2/R/lib/SparkR/tests/testthat/te

In [4]:
# download data csv
!wget https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/precipitation-readings.csv
!wget https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/stations-Ostergotland.csv
!wget https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/stations.csv
!wget https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/temperature-readings-small.csv
!wget https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/temperature-readings.csv

--2023-04-15 11:57:57--  https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/precipitation-readings.csv
Resolving files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com (files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com)... 47.254.186.72
Connecting to files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com (files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com)|47.254.186.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661060231 (630M) [text/csv]
Saving to: ‘precipitation-readings.csv’

precipitation-readi 100%[===================>] 630.44M  28.8MB/s    in 26s     

2023-04-15 11:58:24 (23.9 MB/s) - ‘precipitation-readings.csv’ saved [661060231/661060231]

--2023-04-15 11:58:24--  https://files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com/stations-Ostergotland.csv
Resolving files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com (files-fra-2052bd8a99a5.oss-eu-central-1.aliyuncs.com)... 47.254.186.72
Connecting to files-fra-2052bd8a99a5.oss-eu-central-1

In [83]:
from pyspark import SparkContext

sc = SparkContext(appName = "Lab")

In [85]:
# Assignment 1
!rm -rf ./output/A1

temperature_file = sc.textFile("temperature-readings.csv")
lines = temperature_file.map(lambda line: line.split(";"))

# (key, value) = (year,temperature)
year_temperature = lines.map(lambda x: (x[1][0:4], float(x[3])))

#filter
year_temperature = year_temperature.filter(lambda x: int(x[0])>=1950 and int(x[0])<=2014)

#Get max
max_temperatures = year_temperature.reduceByKey(lambda a,b: max(a, b))
max_temperatures = max_temperatures.sortBy(ascending = False, keyfunc=lambda k: k[1])

#Get min
min_temperatures = year_temperature.reduceByKey(lambda a,b: min(a, b))
min_temperatures = min_temperatures.sortBy(ascending = False, keyfunc=lambda k: k[1])

# print(max_temperatures.collect())
# print("____________________________________________")
# print(min_temperatures.collect())

# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder
max_temperatures.saveAsTextFile("./output/A1/max/")
min_temperatures.saveAsTextFile("./output/A1/min/")



In [87]:
# Assigment 2-1
!rm -rf ./output/A2/1/

temperature_file = sc.textFile("temperature-readings.csv")
lines = temperature_file.map(lambda line: line.split(";"))

# (key, value) = ((year,month),temperature)
year_temperature = lines.map(lambda x: ((x[1][0:4],x[1][5:7]), float(x[3])))

#filter
year_temperature = year_temperature.filter(lambda x: int(x[0][0])>=1950 and int(x[0][0])<=2014 and x[1]>10)

#(key, value) = ((year,month),count)
year_month_count=year_temperature.map(lambda x:(x[0],1))

#Get counts
year_month_counts = year_month_count.reduceByKey(lambda a,b: a+1)

# print(year_month_counts.collect())

# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder
year_month_counts.saveAsTextFile("./output/A2/1/")



In [90]:
# Assigment 2-2
!rm -rf ./output/A2/2/

temperature_file = sc.textFile("temperature-readings.csv")
lines = temperature_file.map(lambda line: line.split(";"))

# (key, value) = ((year,month,station),temperature)
year_temperature = lines.map(lambda x: ((x[1][0:4],x[1][5:7],x[0]), float(x[3])))\
                    .filter(lambda x: int(x[0][0])>=1950 and int(x[0][0])<=2014 and x[1]>10)

# Reduce the value
year_month_sta_count=year_temperature.reduceByKey(lambda x,y:max(x,y))\
                    .map(lambda x:((x[0][0],x[0][1]),x[1]))\
                    .countByKey()         #Count the number of each key and return a dict

# print(year_month_sta_count.items())
# dict->items->list->RDD->save
sc.parallelize(list(year_month_sta_count.items())).saveAsTextFile("./output/A2/2/")


In [91]:
#According to the requirements at the beginning of the file, the algorithm of average temperature needs to be changed.
# Assigment 3
!rm -rf ./output/A3/

temperature_file = sc.textFile("temperature-readings.csv")
lines = temperature_file.map(lambda line: line.split(";"))

# (key, value) = ((year,month,date,station),temperature)
year_temperature = lines.map(lambda x: ((x[1][0:4],x[1][5:7],x[1][8:],x[0]), float(x[3])))\
                    .filter(lambda x: int(x[0][0])>=1960 and int(x[0][0])<=2014)

# print(year_temperature.collect())

# Compute the month average temperature for each station
#daily max temperature
max_sta_temperature=year_temperature.reduceByKey(lambda x,y:max(x,y))
#daily min temperature
min_sta_temperature=year_temperature.reduceByKey(lambda x,y:min(x,y))
#Average monthly temperature
average_sta_temperature=max_sta_temperature.join(min_sta_temperature)\
                        .map(lambda x:((x[0][0],x[0][1],x[0][3]),(x[1][0]+x[1][1])/2))\
                        .groupByKey()\
                        .mapValues(lambda x:sum(x)/len(x))
'''
groupByKey: Group the values for each key in the RDD into a single sequence. Hash-partitions the resulting RDD with numPartitions partitions.
mapValues: Pass each value in the key-value pair RDD through a map function without changing the keys; this also retains the original RDD’s partitioning.
'''

print(average_sta_temperature.collect())

# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder
average_sta_temperature.saveAsTextFile("./output/A3/")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [93]:
# Assigment 4
!rm -rf ./output/A4/

temperature_file = sc.textFile("temperature-readings.csv")
precipitation_file=sc.textFile("precipitation-readings.csv")
temperature_lines = temperature_file.map(lambda line: line.split(";"))
precipitation_lines = precipitation_file.map(lambda line: line.split(";"))

#Temperature
# (key, value) = (station,temperature)
station_temperature = temperature_lines.map(lambda x: (x[0], float(x[3])))\
                    .reduceByKey(lambda x,y:max(x,y))\
                    .filter(lambda x:x[1]>25 and x[1]<30)

#Precipitation
# (key, value) = ((year,month,date,station),precipitation)
station_precipitation= precipitation_lines.map(lambda x: ((x[1][0:4],x[1][5:7],x[1][8:],x[0]), float(x[3])))\
                    .reduceByKey(lambda x,y:x+y)\
                    .map(lambda x:(x[0][3],x[1]))\
                    .reduceByKey(lambda x,y:max(x,y))\
                    .filter(lambda x:x[1]>100 and x[1]<200)

#Join
conbine_data=station_temperature.join(station_precipitation)
'''
Return an RDD containing all pairs of elements with matching keys in self and other.
Each pair of elements will be returned as a (k, (v1, v2)) tuple, where (k, v1) is in self and (k, v2) is in other.
'''

# print(conbine_data.collect())
# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder
conbine_data.saveAsTextFile("./output/A4/")

[]


In [95]:
# Assigment 5
!rm -rf ./output/A5/

precipitation_file=sc.textFile("precipitation-readings.csv")
ost_station_file=sc.textFile("stations-Ostergotland.csv")
precipitation_lines = precipitation_file.map(lambda line: line.split(";"))
ost_station_lines_bc=ost_station_file.map(lambda line: line.split(";")).map(lambda x:int(x[0]))

# broadcast
bc=sc.broadcast(ost_station_lines_bc.collect())

# (key, value) = ((year,month,station),precipitation)
ost_average_monthly_precipitation=precipitation_lines.filter(lambda x:(int(x[0])in bc.value) and \
                                    int(x[1][0:4])>=1993 and int(x[1][0:4])<=2016)\
        .map(lambda x:((x[1][0:4],x[1][5:7],x[0]),float(x[3])))\
        .reduceByKey(lambda x,y:x+y)\
        .map(lambda x:((x[0][0],x[0][1]),x[1]))\
        .groupByKey()\
        .mapValues(lambda x:sum(x)/len(x))

print(ost_average_monthly_precipitation.collect())

# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder
ost_average_monthly_precipitation.saveAsTextFile("./output/A5/")


[(('2016', '07'), 0.0), (('2016', '06'), 47.6625), (('2016', '05'), 29.250000000000004), (('2016', '04'), 26.90000000000001), (('2016', '03'), 19.962500000000002), (('2016', '02'), 21.5625), (('2016', '01'), 22.325), (('2015', '12'), 28.925), (('2015', '11'), 63.88750000000002), (('2015', '10'), 2.2625), (('2015', '09'), 101.3), (('2015', '08'), 26.987499999999997), (('2015', '07'), 119.09999999999995), (('2015', '06'), 78.66250000000001), (('2015', '05'), 93.22499999999998), (('2015', '04'), 15.3375), (('2015', '03'), 42.61250000000001), (('2015', '02'), 24.825000000000003), (('2015', '01'), 59.11250000000004), (('2014', '12'), 35.46250000000001), (('2014', '11'), 52.42500000000005), (('2014', '10'), 72.13749999999999), (('2014', '09'), 48.45000000000001), (('2014', '08'), 90.81249999999997), (('2014', '07'), 22.9875), (('2014', '06'), 75.13750000000002), (('2014', '05'), 58.00000000000001), (('2014', '04'), 31.762500000000003), (('2014', '03'), 36.56250000000001), (('2014', '02'), 43

Both broadcast variables and join operations are common ways to process data in Spark, and their main differences lie in their applicable scenarios and the nature of operations.

Broadcast variables are usually suitable for a small set of read-only data, such as configuration files, dictionaries, etc. Broadcast variables can cache these datasets in each executor's memory for use during task execution. In this way, it is possible to avoid transmitting a large amount of data on the network, and to improve the efficiency of task execution.

In contrast, the join operation is usually suitable for the connection between two or more large data sets. When performing a join operation, Spark needs to transmit and process data between different nodes, which may cause large network overhead and performance bottlenecks.

In terms of performance, using broadcast variables can avoid data transmission and shuffle operations, so broadcast variables are usually faster than join operations. However, broadcast variables are suitable for small data sets, and if the data set is large, broadcast variables can use a lot of memory and may affect performance.

In general, broadcast variables are suitable for small read-only datasets, while join operations are suitable for connections between two or more large datasets. In practical applications, appropriate operations should be selected according to the size and characteristics of the dataset.

In [82]:
sc.stop()